# `F = ma` - with activation - extra layer

Some 3D funcs that were useful.

In [ ]:
import torch
from torch import nn
from torch.optim import Adam
from matplotlib import pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np

## Model

Keeping this extremely simple

In [ ]:
class LinearLayer(nn.Module):
    def __init__(self, in_feats, out_feats, use_act=True, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.linear = nn.Linear(in_feats, out_feats)
        self.act = nn.ReLU()
        self.use_act = use_act

    def forward(self, x):
        x = self.linear(x)
        if self.use_act:
            x = self.act(x)
        return x


class Net(nn.Module):
    def __init__(self, use_act=True, dim=32, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.layers = nn.Sequential(LinearLayer(2, dim, use_act=use_act), LinearLayer(dim, 1, use_act=False))

    def forward(self, x):
        return self.layers(x)

## Data

Going to sample $x_1$ (mass) and $x_2$ (acceleration) such that their product is uniformly distributed.

In [ ]:
eps = 0.5

Y = torch.rand(10000)
X = torch.empty(10000, 2)
X[:, 0] = Y / (torch.rand(10000) * (1 - eps) + eps)
X[:, 1] = Y / X[:, 0]

# need to randomly swap x1 and x2 so that they're identically distributed - can do this because their product is commutative
mask = torch.rand(10000) < 0.5
swap_vals = X[:, 0][mask]
X[:, 0][mask] = X[:, 1][mask]
X[:, 1][mask] = swap_vals

assert torch.allclose(X[:, 0] * X[:, 1], Y)

Y = Y[:, None]
X.shape, Y.shape

In [ ]:
fig, ax = plt.subplots()

ax.hist(X[:, 0], alpha=0.5, density=True, label="mass")
ax.hist(X[:, 1], alpha=0.5, density=True, label="acceleration")
ax.hist(Y[:, 0], alpha=0.5, density=True, label="force")
ax.legend()

In [ ]:
s_inds = np.random.permutation(range(X.shape[0]))  # shuffled indices

X_train = X[s_inds[:8000]]
Y_train = Y[s_inds[:8000]]
X_valid = X[s_inds[8000:]]
Y_valid = Y[s_inds[8000:]]

X_train.shape, Y_train.shape, X_valid.shape, Y_valid.shape

## Training

I'll just do full gradient descent to keep things simple.

In [ ]:
N = 2000  # number of epochs

model = Net(use_act=True, dim=2)
loss_func = nn.MSELoss()
optimiser = Adam(model.parameters(), lr=1e-3)
log = []

for i in tqdm(range(N)):
    log_sample = {}

    # Training update
    model.train()
    model.zero_grad()
    Y_hat = model(X_train)
    loss = loss_func(Y_hat, Y_train)
    log_sample["train_loss"] = float(loss.detach())
    loss.backward()
    optimiser.step()

    # Validation set
    model.eval()
    Y_hat = model(X_valid)
    loss = loss_func(Y_hat, Y_valid)
    log_sample["valid_loss"] = float(loss.detach())

    log.append(log_sample)

df = pd.DataFrame(log)

## Results

In [ ]:
def plot_loss(train_loss, valid_loss):
    _, ax = plt.subplots()

    ax.plot(train_loss, label="train")
    ax.plot(valid_loss, label="valid")
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Loss")
    ax.legend()


def get_preds(model):
    model.eval()

    with torch.inference_mode():
        out = model(X_valid)

    y_preds = out.flatten().numpy()
    y_targs = Y_valid.flatten().numpy()

    return y_preds, y_targs


def get_valid_loss(model, loss_func):
    model.eval()

    with torch.inference_mode():
        out = model(X_valid)

    return loss_func(out, Y_valid)


def plot_results(y_preds, y_targs):
    _, ax = plt.subplots()

    ax.scatter(y_preds, y_targs, s=4, label="predictions")
    ax.plot(y_targs, y_targs, color="salmon", ls="--", lw=0.5, label="perfect accuracy")
    ax.set(xlabel="y_preds", ylabel="y_targs")
    ax.legend()

In [ ]:
plot_loss(df["train_loss"], df["valid_loss"])

In [ ]:
y_preds, y_targs = get_preds(model)

In [ ]:
get_valid_loss(model, loss_func)

In [ ]:
plot_results(y_preds, y_targs)

## Introspection

In [ ]:
model

In [ ]:
for name, param in model.named_parameters():
    print(name, "\n", param.data, "\n")

### Inspect intermediate values

Going to go operation by operation to see if I can see what's going on.

input ($\rightarrow$ prebias_preacts $\rightarrow$ preacts $\rightarrow$ acts) ($\rightarrow$ prebias $\rightarrow$ output)

(with the layers grouped in brackets)

In [ ]:
SD = model.state_dict()

In [ ]:
def model_breakdown(x1, x2, print_steps=True, return_steps=False):
    x1, x2 = float(x1), float(x2)

    x = torch.tensor([[x1, x2]]).T
    prebias_preacts = torch.matmul(SD["layers.0.linear.weight"], x)
    preacts = prebias_preacts[:, 0] + SD["layers.0.linear.bias"]
    acts = torch.nn.functional.relu(preacts)
    prebias = torch.matmul(SD["layers.1.linear.weight"], acts)
    output = prebias + SD["layers.1.linear.bias"]

    if print_steps:
        print("Input:", "\n", x, "\n")

        print("================ Layer 0 ================", "\n")
        print("Weight:", "\n", SD["layers.0.linear.weight"], "\n")
        print("Result:", "\n", prebias_preacts, "\n")
        print("Bias:", "\n", SD["layers.0.linear.bias"], "\n")
        print("Result:", "\n", preacts, "\n")
        print("Activations:", "\n", acts, "\n")

        print("================ Layer 1 ================", "\n")
        print("Weight:", "\n", SD["layers.1.linear.weight"], "\n")
        print("Result:", "\n", prebias, "\n")
        print("Bias:", "\n", SD["layers.1.linear.bias"], "\n")
        print("Output:", "\n", output, "\n")

        print(f"y_true = {x1*x2:.3f}")

    if return_steps:
        return prebias_preacts, preacts, acts, prebias, output
    
def plot_model_breakdown(x1, x2, ax=None, legend=True, scatter=True):
    %matplotlib ipympl

    if not ax:
        fig = plt.figure()
        ax = plt.axes(projection="3d")

    input = (x1, x2)

    prebias_preacts, preacts, acts, prebias, output = model_breakdown(*input, print_steps=False, return_steps=True)

    x = [[input[0], input[1], 0], prebias_preacts.flatten().tolist(), preacts.flatten().tolist(), acts.flatten().tolist(), [float(prebias),0,0], [float(output),0,0]]
    x = np.asarray(x)

    # Plot the surface
    # ax.plot_surface(X, Y, Z, cmap="viridis", edgecolor="none")
    ax.scatter(*x[0], alpha=1.0, label='input', color='mediumseagreen')
    if scatter:
        ax.scatter(*x[1], marker='x', alpha=1.0, label='prebias_preacts', color='darkgray')
        ax.scatter(*x[2], marker='x', alpha=1.0, label='preacts', color='mediumpurple')
        ax.scatter(*x[3], marker='x', alpha=1.0, label='acts', color='turquoise')
        ax.scatter(*x[4], marker='x', alpha=1.0, label='prebias', color='hotpink')
    ax.scatter(*x[5], marker='x', alpha=1.0, label='output', color='lightcoral')

    ax.plot(*x.T, lw=0.5)
    ax.set(xlabel='component 0', ylabel='component 1', zlabel='component 2')
    if legend:
        ax.legend();

In [ ]:
model_breakdown(0.5, 0.5)

In [ ]:
fig = plt.figure()
ax = plt.axes(projection="3d")

N = 2
pairs = np.concatenate(np.stack(np.meshgrid(np.linspace(0, 1, N), np.linspace(0, 1, N))).T)

plot_model_breakdown(*pairs[0], ax, scatter=False)
for pair in pairs[1:]:
    plot_model_breakdown(*pair, ax, legend=False, scatter=False)

Not a very interpretable diagram:

In [ ]:
SD = model.state_dict()
weight = SD["layers.0.linear.weight"].numpy()
bias = SD["layers.0.linear.bias"].numpy()

fig, ax = plt.subplots()

sc = ax.scatter(*weight.T, c=bias, cmap="coolwarm")

# Add a colorbar
cbar = fig.colorbar(sc, ax=ax, label="bias")

ax.set(xlabel="Component 0", ylabel="Component 1")

ax.axvline(0, color="k", linestyle="--", linewidth=0.5)
ax.axhline(0, color="k", linestyle="--", linewidth=0.5)

How's this actually going to _transform_ the input?

In [ ]:
def get_preds_range(x2=0.5):
    # True solution
    x1 = np.linspace(0, 1, 50)
    x2 = np.repeat(x2, 50)
    y = x1 * x2

    # Predicted solution
    with torch.inference_mode():
        x = torch.stack([torch.as_tensor(x1), torch.as_tensor(x2)]).T.float()
        y_hat = model(x)
        y_hat = y_hat.detach().numpy()

    return y_hat, y


def get_preds_multi_range():
    x2s = np.linspace(0, 1, 11)
    rows = []
    for x2 in x2s:
        row = {"x2": x2}
        row["y_hat"], row["y"] = get_preds_range(x2=x2)
        rows.append(row)
    return rows

In [ ]:
results = get_preds_multi_range()

Fixing $x_2$ and varying $x_1$ to isolate behaviour:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

for row in results:
    ax.plot(row["y_hat"], row["y"], lw=1, label=f"x2 = {row['x2']:.2f}")

ax.plot([0, 1], [0, 1], color="salmon", lw=2, ls="--", label="perfect accuracy")
ax.set(xlabel="y_preds", ylabel="y_targs")
ax.legend()